# 네이버 지도 리뷰 크롤링

In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from user_agent import generate_user_agent
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.request import Request
import time
import pandas as pd
import re
import random

* 서울시 반려동물 동반 가능 시설 데이터 Load

In [2]:
df = pd.read_csv("../data/preprocessed_per_culture_site.csv", index_col=0)
df.head()

,시설명,카테고리,주소,위도,경도
0,100세약국,동물약국,서울특별시 영등포구 당산동6가 217-4,37.533260,126.903741
1,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
2,1층메디컬약국,동물약국,서울특별시 금천구 시흥동 889-8,37.453043,126.901712
3,1층이화약국,동물약국,서울특별시 성동구 하왕십리동 1070,37.566590,127.024082
4,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196


* 동물약국은 제외한다

In [4]:
df = df[df["카테고리"]!="동물약국"]
df.reset_index(drop=True, inplace=True)
df.head()

,시설명,카테고리,주소,위도,경도
0,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417
1,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196
2,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063
3,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867
4,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256


* 추후 리뷰 크롤링을 위해 고유 id값을 담을 컬럼 추가

In [3]:
df["인프라id"] = ""
df.head()

,시설명,카테고리,주소,위도,경도,인프라id
0,100세약국,동물약국,서울특별시 영등포구 당산동6가 217-4,37.533260,126.903741,
1,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417,
2,1층메디컬약국,동물약국,서울특별시 금천구 시흥동 889-8,37.453043,126.901712,
3,1층이화약국,동물약국,서울특별시 성동구 하왕십리동 1070,37.566590,127.024082,
4,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196,


* 데이터프레임에 담긴 시설을 네이버지도에서 검색하여 고유 id 값을 가져와서 인프라id 컬럼에 추가

In [ ]:
options = Options()
options.add_argument('--disable-images')
options.add_experimental_option("prefs", {'profile.managed_default_content_settings.images': 2})
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
id_src = ""

for _, row in df.iterrows():
    url = "https://map.naver.com/p/search/" + str(row["시설명"]).replace("24시", "") + " " + row["주소"] + "?c=15.00,0,0,0,dh"

    driver.get(url)

    try:
        id_src = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "entryIframe"))).get_attribute("src")
    except:
        continue

    id = re.findall("\/(\d+)?", id_src)[3]
    df.loc[df["시설명"]==row["시설명"], "인프라id"] = id

driver.quit()

* 인프라id가 잘 담겼는지 확인

In [32]:
df.head()

,시설명,카테고리,주소,위도,경도,인프라id
0,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,36399514
1,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,1347656288
2,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,87231883
3,24시 산들산들 동물병원,동물병원,서울특별시 마포구 아현동 424-14,37.550975,126.956497,37371383
4,24시 서울 탑 동물병원,동물병원,서울특별시 양천구 신정동 1182-11,37.517743,126.853643,715618768


* 해당 데이터 저장

In [ ]:
df.to_csv("../data/naver_review_data.csv")

In [31]:
df = pd.read_csv("../data/naver_review_data.csv", index_col=0, dtype={'인프라id':object})
df.reset_index(drop=True, inplace=True)

df.head()

,시설명,카테고리,주소,위도,경도,인프라id
0,100코달리 와인바,카페,서울특별시 중구 신당동 292-152,37.563305,127.016417,NaN
1,21세기동물병원,동물병원,서울특별시 용산구 보광동 259-1,37.528316,126.999196,NaN
2,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,36399514
3,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,1347656288
4,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,87231883


* 인프라id에 Nan이 들어간 행은 네이버 지도에서 검색되지 않는 시설이다(필터링)

In [32]:
df = df[df["인프라id"].isna() == False]
df.reset_index(drop=True, inplace=True)
df.head()

,시설명,카테고리,주소,위도,경도,인프라id
0,24시 강서 젠틀리동물의료센터,동물병원,서울특별시 강서구 마곡동 747,37.559040,126.820063,36399514
1,24시 굿케어동물의료센터,동물병원,서울특별시 관악구 봉천동 1673-21,37.480000,126.956867,1347656288
2,24시 글로리 동물병원,동물병원,서울특별시 금천구 독산동 145-13,37.476744,126.898256,87231883
3,24시 산들산들 동물병원,동물병원,서울특별시 마포구 아현동 424-14,37.550975,126.956497,37371383
4,24시 서울 탑 동물병원,동물병원,서울특별시 양천구 신정동 1182-11,37.517743,126.853643,715618768


* 셀레니움에서 리뷰 더보기 버튼을 눌러주는 함수

In [33]:
def morebtn_click(proc_driver):
    action = ActionChains(proc_driver)

    while True:
        try:
            more_btn = WebDriverWait(proc_driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "TeItc")))
            proc_driver.execute_script("arguments[0].click();", more_btn)
        except:
            print("click done....")
            return True
        
    return False

* 이모지를 걸러내기 위한 함수

In [34]:
def remove_emojis(text):
    pattern = r'[^0-9a-zA-Z가-힣!@#\$%\^&\*\(\)\-_=+\{\}\[\]:;"\'<>,\.\/?`~ ]+'
    filter_text = re.sub(pattern, '', text)

    return filter_text

* 리뷰를 가져오는 함수

In [35]:
def get_review(html):
    review_list = []
    """
    li_list = driver.find_elements(By.CLASS_NAME, "pui__X35jYm.place_apply_pui.EjjAW")

    for comment in li_list:
        try:
            comment_tmp = comment.find_element(By.CLASS_NAME, "pui__vn15t2").text
        except Exception as e:
            print("에러 발생")
            continue
        if not comment_tmp == "":
            comment_tmp = comment_tmp.replace("\n", " ")
            comment_tmp = comment_tmp.replace("더보기", "")
            review_list.append(remove_emojis(comment_tmp))
    """

    req = Request(url, headers={"User-Agent":generate_user_agent()})
    soup = BeautifulSoup(html, "html.parser")

    review_li = soup.find("div", "place_section k1QQ5")
    for row in review_li.find_all("li", "pui__X35jYm place_apply_pui EjjAW"):
        review_list.append(remove_emojis(row.find("div", "pui__vn15t2").get_text()))
            
    print("리뷰수 : " + str(len(review_list)))
    return review_list

In [36]:
review_dict = {}

* 데이터프레임에 담긴 시설 정보를 토대로 네이버 플레이스에서 리뷰를 크롤링해서 딕셔너리에 저장

In [48]:
options = Options()
options.add_argument('--disable-images')
options.add_experimental_option("prefs", {'profile.managed_default_content_settings.images': 2})
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument(f"user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36")
options.add_argument('--headless') 

driver = webdriver.Chrome(service=Service("../driver/chromedriver"))
driver.delete_all_cookies()

for _, row in tqdm(df.iloc[:56].iterrows(), total=len(df.iloc[:56].index)):
    url = "https://pcmap.place.naver.com/place/" + str(row["인프라id"]) + "/review/visitor"
    driver.get(url)

    if "Too Many Requests" in driver.page_source:
         print(driver.page_source)
         break

    print("[" + row["시설명"] + "] processing...")

    try:
     if morebtn_click(driver):
          review_list = get_review(driver.page_source)
    except:
        print(row["시설명"], "리뷰 탭 없음")
        continue

    review_dict[row["시설명"]] = review_list

    time.sleep(random.uniform(0.1, 0.3))

driver.quit()

  0%|          | 0/56 [00:00<?, ?it/s]

<html><head><title>429 Too Many Requests</title></head>
<body bgcolor="white">
<center><h1>429 Too Many Requests</h1></center>
<hr><center>nginx/1.14.2</center>








</body></html>


* 딕셔너리에 저장된 시설에 대한 리뷰를 풀어서 데이터프레임으로 구성

In [44]:
data = []

for name, list in review_dict.items():
    for review in list:
        data.append({"시설명":name, "리뷰":review})

review_df = pd.DataFrame(data)
review_df.head()

,시설명,리뷰
0,강아지분양 미니몽 은평점,처음에는 가격대가 부담스러워서 고민했는데 애기가 이쁘고 얌전해서 데려왔어요~ 사장님...
1,강아지분양 미니몽 은평점,은평구 강아지분양 검색하다가 리뷰가 너무 좋아서 찾아간 미니몽!!! 너무너무 이쁜 ...
2,강아지분양 미니몽 은평점,콩이 너무 착한가격과 친절한 서비스로 데리고왔습니다 여기진짜 정직하고 괜찮은 곳입니다
3,강아지분양 미니몽 은평점,너무 이쁜 애기 비숑을 새 가족으로 맞이했어요 밤이고 낮이고 한번 더 보겠다고 방문...
4,강아지분양 미니몽 은평점,미니몽에서 분양한얼굴도 이쁘고 활발한 푸들이에요~~아가 이름은 봄처럼 아름답게 살라...


* 리뷰가 공백인 경우를 필터링(사진만 올렸거나 이모지만 작성하여 필터링된 경우)

In [45]:
review_df = review_df[review_df["리뷰"] != ""]
review_df.reset_index(drop=True, inplace=True)
review_df.head()

,시설명,리뷰
0,강아지분양 미니몽 은평점,처음에는 가격대가 부담스러워서 고민했는데 애기가 이쁘고 얌전해서 데려왔어요~ 사장님...
1,강아지분양 미니몽 은평점,은평구 강아지분양 검색하다가 리뷰가 너무 좋아서 찾아간 미니몽!!! 너무너무 이쁜 ...
2,강아지분양 미니몽 은평점,콩이 너무 착한가격과 친절한 서비스로 데리고왔습니다 여기진짜 정직하고 괜찮은 곳입니다
3,강아지분양 미니몽 은평점,너무 이쁜 애기 비숑을 새 가족으로 맞이했어요 밤이고 낮이고 한번 더 보겠다고 방문...
4,강아지분양 미니몽 은평점,미니몽에서 분양한얼굴도 이쁘고 활발한 푸들이에요~~아가 이름은 봄처럼 아름답게 살라...


* 현재까지 리뷰 데이터 저장

In [47]:
review_df.to_csv("../data/naver_review.csv", encoding="utf-8")

* 나눠서 저장한 리뷰데이터를 하나로 합치기

In [3]:
import pandas as pd

review1 = pd.read_csv("../data/naver_review.csv")
review1

,Unnamed: 0,시설명,리뷰
0,0,강아지분양 미니몽 은평점,처음에는 가격대가 부담스러워서 고민했는데 애기가 이쁘고 얌전해서 데려왔어요~ 사장님...
1,1,강아지분양 미니몽 은평점,은평구 강아지분양 검색하다가 리뷰가 너무 좋아서 찾아간 미니몽!!! 너무너무 이쁜 ...
2,2,강아지분양 미니몽 은평점,콩이 너무 착한가격과 친절한 서비스로 데리고왔습니다 여기진짜 정직하고 괜찮은 곳입니다
3,3,강아지분양 미니몽 은평점,너무 이쁜 애기 비숑을 새 가족으로 맞이했어요 밤이고 낮이고 한번 더 보겠다고 방문...
4,4,강아지분양 미니몽 은평점,미니몽에서 분양한얼굴도 이쁘고 활발한 푸들이에요~~아가 이름은 봄처럼 아름답게 살라...
...,...,...,...
70987,70987,ZOO동물병원,항상가는곳이에요 원장님도 친절하세요!!
70988,70988,ZOO동물병원,서울에 출장중이라 말씀드려서 그런가 다시는 안올사람 처럼 보였는지 너무 퉁명하게 말...
70989,70989,ZOO동물병원,고양이를 세 마리 키우고 있어서 여러 동물 병원 다녀봤는데 지금까지 꾸준하게 한 곳...
70990,70990,ZOO동물병원,친절하세요


In [4]:
review2 = pd.read_csv("../data/naver_review2.csv", )
review2

,Unnamed: 0,시설명,리뷰
0,0,24시 강서 젠틀리동물의료센터,뼈다귀 맛집
1,1,24시 강서 젠틀리동물의료센터,굿
2,2,24시 강서 젠틀리동물의료센터,애기아파서 들렀는데 친절하게 잘 상담해주셨어요
3,3,24시 강서 젠틀리동물의료센터,야간에도 진료를 잘봐줘요
4,4,24시 강서 젠틀리동물의료센터,좋아요
...,...,...,...
14814,14814,강아지대학로,귀여워요!!
14815,14815,강아지대학로,친절해요
14816,14816,강아지대학로,좋아요
14817,14817,강아지대학로,깨끗하고 사장님 친절하신 강아지카페. 강아지들도 순둥이들


In [13]:
naver_review = pd.concat([review2, review1], ignore_index=False)
naver_review.drop(columns=["Unnamed: 0"], inplace=True)
naver_review

,시설명,리뷰
0,24시 강서 젠틀리동물의료센터,뼈다귀 맛집
1,24시 강서 젠틀리동물의료센터,굿
2,24시 강서 젠틀리동물의료센터,애기아파서 들렀는데 친절하게 잘 상담해주셨어요
3,24시 강서 젠틀리동물의료센터,야간에도 진료를 잘봐줘요
4,24시 강서 젠틀리동물의료센터,좋아요
...,...,...
70987,ZOO동물병원,항상가는곳이에요 원장님도 친절하세요!!
70988,ZOO동물병원,서울에 출장중이라 말씀드려서 그런가 다시는 안올사람 처럼 보였는지 너무 퉁명하게 말...
70989,ZOO동물병원,고양이를 세 마리 키우고 있어서 여러 동물 병원 다녀봤는데 지금까지 꾸준하게 한 곳...
70990,ZOO동물병원,친절하세요


* 합친 리뷰 데이터 CSV 저장

In [14]:
naver_review.to_csv("../data/final_naver_review.csv", encoding="utf-8")

* 리뷰 데이터 DB에 저장

In [52]:
import mysql.connector

def connect_db():
    try:
        conn = mysql.connector.connect(
            host = "database-2.c3iym8yog7ht.ap-northeast-2.rds.amazonaws.com",
            user = "pethub",
            password = "addinedu5",
            database = "pethub"
        )

        if conn.is_connected():
            print("success connect...")
    except Exception as e:
        print("failed connect...", e)

    return conn

def execute_query(conn, query, param=None):
    if not conn.is_connected():
        print("not connected db...")
        return
    
    cursor = conn.cursor()
    try:
        cursor.execute(query, param)

        conn.commit()
    except Exception as e:
        print("failed excute query...", e)
        return
    finally:
        cursor.close()

    return True

def fetch_all(conn, query):
    if not conn.is_connected():
        print("not connected db...")
        return
    
    cursor = conn.cursor(buffered=True)
    try:
        cursor.execute(query)

        result = cursor.fetchall()

        return result
    except Exception as e:
        print("failed fetch...", e)
        return
    finally:
        cursor.close()

In [53]:
conn = connect_db()

success connect...


In [55]:
for _, row in review_df.loc[:10000].iterrows():
    query = "insert into review(infra_id, rating, comment) values(%s, %s, %s)"
    execute_query(conn, query, (1, 4.0, row["리뷰"]))